Importing required libraries

In [3]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path
import csv
import pandas as pd

Function to extract footnotes and save them to a CSV file.

In [8]:
# Define the namespace used in the XML
namespace = {'pc': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}

# Find all TextRegion elements with 'footnote' in their custom attribute
footnote_data = {}
all_footnotes = []
field_names = ['File Name', 'Footnote']
def extract_footnotes(filepath):
    xml_files_path = Path(filepath).rglob('*.xml')
    for file in xml_files_path:
        xml_file = str(file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for page in root.findall('.//pc:Page', namespace):
            page_name = page.attrib.get('imageFilename') 
            footnotes_text = []
            current_footnote = []
            for text_region in root.findall(".//pc:TextRegion", namespace):
                custom_attr = text_region.attrib.get('custom', '')
                if 'structure {type:footnote-continued;}' in custom_attr:
                        current_footnote.append('(cotd.)')
                        # Extract TextLine elements within the TextRegion
                        for text_line in text_region.findall(".//pc:TextLine", namespace):
                        # Get the Unicode content
                            unicode_elem = text_line.find(".//pc:Unicode", namespace)
                            if unicode_elem is not None:
                                text = unicode_elem.text.strip()
                                current_footnote.append(text)
                        footnotes_text.append(' '.join(current_footnote))
                        all_footnotes.append(' '.join(current_footnote))
                        current_footnote = []
                if 'structure {type:footnote;}' in custom_attr:
                    if 'structure {type:footnote-continued;}' in custom_attr:
                        current_footnote.append('(cotd.)')
                        # Extract TextLine elements within the TextRegion
                        for text_line in text_region.findall(".//pc:TextLine", namespace):
                        # Get the Unicode content and add to existing footnote
                            unicode_elem = text_line.find(".//pc:Unicode", namespace)
                            if unicode_elem is not None:
                                text = unicode_elem.text.strip()
                                current_footnote.append(text)
                        # Save current footnote and clear out content
                        footnotes_text.append(' '.join(current_footnote))
                        all_footnotes.append(' '.join(current_footnote))
                        current_footnote = []
                    # Extract TextLine elements within the TextRegion
                    for text_line in text_region.findall(".//pc:TextLine", namespace):
                        # Get the Unicode content
                        unicode_elem = text_line.find(".//pc:Unicode", namespace)
                        if unicode_elem is not None:
                            text = unicode_elem.text.strip()
                            # Check if it is a new footnote
                            if text.startswith('*') or text.startswith('†') or text.startswith('‡'):
                                # Save any existing footnote
                                if current_footnote:
                                    footnotes_text.append(' '.join(current_footnote))
                                    all_footnotes.append(' '.join(current_footnote))
                                    # Clear out current footnote data
                                    current_footnote = []
                                # Update current_footnote with new footnote text
                                current_footnote.append(text)
                            # If it not a new footnote, continue adding to existing footnote text
                            else:
                                current_footnote.append(text)
                    # Add last footnote
                    if current_footnote:
                        footnotes_text.append(' '.join(current_footnote))
                        all_footnotes.append(' '.join(current_footnote))
                # Save footnotes information to dictionary
                if footnotes_text:
                    footnote_data.update({
                    page_name: footnotes_text})

    # Write the footnotes to a CSV file
    output_csvfile = Path(filepath) / 'extracted_footnotes.csv'
    with open(output_csvfile, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(field_names)

        for file_name, footnotes in footnote_data.items():
            for footnote in footnotes:
                writer.writerow([file_name, footnote])
    
    # Footnotes were reordered and the nine continuing footnotes were added to the previous footnote using OpenRefine. Continuing footnotes were also removed as independent records from the data. This new data is saved as ordered_footnotes.csv.
    # Save footnotes from ordered_footnotes.csv to a text file
    output_textfile = ('data/footnotes/the_land_of_the_perumauls_xml_export/files/The_Land_of_the_Perumauls/page/extracted_footnotes.txt')
    ordered_footnotes = pd.read_csv('data/footnotes/the_land_of_the_perumauls_xml_export/files/The_Land_of_the_Perumauls/page/ordered_footnotes.csv')
    with open(output_textfile, 'w', encoding='utf-8') as textfile:
        for footnote in ordered_footnotes['Footnote']:
            textfile.write(footnote + '\n')

    print(f"Footnotes extracted and saved to {output_csvfile} and {output_textfile}.")

extract_footnotes('data/footnotes/the_land_of_the_perumauls_xml_export/files/The_Land_of_the_Perumauls/page')

Footnotes extracted and saved to data/footnotes/the_land_of_the_perumauls_xml_export/files/The_Land_of_the_Perumauls/page/extracted_footnotes.csv and data/footnotes/the_land_of_the_perumauls_xml_export/files/The_Land_of_the_Perumauls/page/extracted_footnotes.txt.
